In [ ]:
%pip install pandas
%pip install ftfy

In [5]:
#Manual fix to specific rows
rowsTowSwap = [229, 230, 431, 515, 609, 610, 611, 1010, 1011, 1012, 1014, 1015, 1016, 1037,]
for i in range(len(rowsTowSwap)):
    rowsTowSwap[i] -=2

rowsToRemove = [10]
for i in range(len(rowsToRemove)):
    rowsToRemove[i] -=2    

In [6]:
import pandas as pd
from ftfy import fix_text
import re

# find game ids inside item ids
find_parenthesis = re.compile(r"^(.+?)\s*\(([^()]*)\), Reward")

#find rows with no items
find_games_noitems = re.compile(r"(.*)\, Reward: .*")

#manual cleaning of game ids starting with brackets like [root] or [premium]
bracketsInGameID = re.compile(r"^\[.*\] ?(.*)")

In [7]:
# get the data
data = pd.read_csv("all_items_gplay.csv")

In [8]:

# create a new dataframe to store the fixed data
fixed = pd.DataFrame(columns=['game_id','item_id','n','median_amount_USD'])

fixedList = []

for index, row in data.iterrows():
    # get the game_id and item_id
    game_id = fix_text(row['game_id'])
    item_id = fix_text(row['item_id'])
    # get the n and median_amount_USD
    number = row['n']
    median_amount = row['median_amount_USD']

    if (find_parenthesis.match(item_id)):
        game_id = find_parenthesis.search(item_id).group(2)
        item_id = find_parenthesis.search(item_id).group(1)

    if (find_games_noitems.match(item_id)):
        game_id = find_games_noitems.search(item_id).group(1)
        item_id = ""
        rowsToRemove.append(index)


    if( bracketsInGameID.match(game_id)):
        game_id = bracketsInGameID.search(game_id).group(1)

    if (index in rowsTowSwap):
        # swap the game_id and item_id in for manual fix
        temp = game_id
        game_id = item_id
        item_id = temp

    fixed_row = pd.Series({'game_id': game_id, 'item_id': item_id, 'n': number, 'median_amount_USD': median_amount})
    # append the new row to the dataframe
    fixedList.append(fixed_row)

fixed = pd.DataFrame(fixedList)



#If needed, remove rows with no items or unusable data
fixedList = []

for index, row in fixed.iterrows():
    # get the game_id and item_id
    game_id = row['game_id']
    item_id = row['item_id']
    # get the n and median_amount_USD
    number = row['n']
    median_amount = row['median_amount_USD']

    # create a new row with the fixed text
    if (index not in rowsToRemove):
        fixed_row = pd.Series({'game_id': game_id, 'item_id': item_id, 'n': number, 'median_amount_USD': median_amount})
        # append the new row to the dataframe
        fixedList.append(fixed_row)

fixed = pd.DataFrame(fixedList)

# sort data
fixed.sort_values(by=['game_id', 'item_id'], inplace=True)
fixed.reset_index(drop=True, inplace=True)

In [9]:
#save the fixed data to a new csv file
fixed.to_csv("all_items_gplay_fixed.csv", index=False)